In [1]:
!pip install pandas --upgrade
!pip install scikit-learn --upgrade
!pip install imbalanced-learn --upgrade
!pip install xgboost --upgrade
!pip install imblearn --upgrade

In [6]:
from sklearn.impute import SimpleImputer

import pandas as pd
import numpy as np

In [18]:
#Baixar o dataset no colab
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/training_dataset.csv

In [7]:
#Lendo o dataset que está no formato csv 

df_training_dataset = pd.read_csv('training_dataset.csv')
df_training_dataset.tail()

,id,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,total_modulos,modulos_iniciados,modulos_finalizados,certificados,categoria
15852,620397030.0,Bacharelado,UFF,Outros,Borracha,0.0,0.0,1.0,8.0,NaN,10.0,NaN,NaN,NaN,perfil6
15853,229931283.0,Bacharelado,NaN,Advogado,NaN,0.0,0.0,1.0,7.0,Linkedin,42.0,17.0,15.0,NaN,perfil5
15854,480674907.0,Tecnólogo,UNIP,Sócio/Dono/Proprietário,NaN,0.0,NaN,1.0,7.0,Outros,30.0,9.0,8.0,0.0,perfil5
15855,559626807.0,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,NaN,226.0,102.0,93.0,1.0,perfil1
15856,743652801.0,NaN,FGV-RJ,NaN,Siderurgica,1.0,1.0,1.0,9.0,Outros,125.0,98.0,97.0,1.0,perfil1


In [8]:
#Detalhes do dataset

df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15857 entries, 0 to 15856
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          13945 non-null  float64
 1   graduacao                   13950 non-null  object 
 2   universidade                13920 non-null  object 
 3   profissao                   13977 non-null  object 
 4   organizacao                 13961 non-null  object 
 5   pretende_fazer_cursos_lit   13989 non-null  float64
 6   interesse_mba_lit           14003 non-null  float64
 7   importante_ter_certificado  13918 non-null  float64
 8   horas_semanais_estudo       13959 non-null  float64
 9   como_conheceu_lit           13915 non-null  object 
 10  total_modulos               13987 non-null  float64
 11  modulos_iniciados           14044 non-null  float64
 12  modulos_finalizados         13924 non-null  float64
 13  certificados                139

In [9]:
#Quantidade de valores únicos

df_training_dataset.nunique()

id                            13945
graduacao                         6
universidade                     21
profissao                        12
organizacao                      11
pretende_fazer_cursos_lit         2
interesse_mba_lit                 2
importante_ter_certificado        1
horas_semanais_estudo             9
como_conheceu_lit                 9
total_modulos                   578
modulos_iniciados               372
modulos_finalizados             339
certificados                     23
categoria                         6
dtype: int64

In [10]:
#Quantidade de valores nulos

print("Valores nulos: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos: 

id                            1912
graduacao                     1907
universidade                  1937
profissao                     1880
organizacao                   1896
pretende_fazer_cursos_lit     1868
interesse_mba_lit             1854
importante_ter_certificado    1939
horas_semanais_estudo         1898
como_conheceu_lit             1942
total_modulos                 1870
modulos_iniciados             1813
modulos_finalizados           1933
certificados                  1878
categoria                        0
dtype: int64



In [11]:
#Substituindo os nulos por 'Outros'

feats = ['profissao', 'universidade', 'graduacao', 'organizacao', 'como_conheceu_lit']

#Substituindo os valores nulos pela categoria 'Outros'
for feat in feats: 
    df_training_dataset[feat] = df_training_dataset[feat].replace(np.nan, 'Outros')

#Removendo os valores que foram perdidos 
df_training_dataset = df_training_dataset.dropna(
    axis='index', 
    how='any', 
    subset=['horas_semanais_estudo','pretende_fazer_cursos_lit', 'interesse_mba_lit', 'importante_ter_certificado', 
            'certificados', 'modulos_finalizados', 'modulos_iniciados'],
)

#Remoção de Colunas
df_training_dataset = df_training_dataset.drop(columns=['id','total_modulos'], inplace=False)

In [12]:
#Reverificando a quantidade de valores nulos depois das transformações

print("Valores nulos: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos: 

graduacao                     0
universidade                  0
profissao                     0
organizacao                   0
pretende_fazer_cursos_lit     0
interesse_mba_lit             0
importante_ter_certificado    0
horas_semanais_estudo         0
como_conheceu_lit             0
modulos_iniciados             0
modulos_finalizados           0
certificados                  0
categoria                     0
dtype: int64



In [13]:
#Detalhes do dataset

df_training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6578 entries, 0 to 15856
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   graduacao                   6578 non-null   object 
 1   universidade                6578 non-null   object 
 2   profissao                   6578 non-null   object 
 3   organizacao                 6578 non-null   object 
 4   pretende_fazer_cursos_lit   6578 non-null   float64
 5   interesse_mba_lit           6578 non-null   float64
 6   importante_ter_certificado  6578 non-null   float64
 7   horas_semanais_estudo       6578 non-null   float64
 8   como_conheceu_lit           6578 non-null   object 
 9   modulos_iniciados           6578 non-null   float64
 10  modulos_finalizados         6578 non-null   float64
 11  certificados                6578 non-null   float64
 12  categoria                   6578 non-null   object 
dtypes: float64(7), object(6)
memory 

In [14]:
df_training_dataset.tail()

,graduacao,universidade,profissao,organizacao,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,como_conheceu_lit,modulos_iniciados,modulos_finalizados,certificados,categoria
15844,Bacharelado,Unesp,SEM EXPERIÊNCIA,Oleo e Gas,0.0,0.0,1.0,6.0,Outros,39.0,32.0,0.0,perfil2
15847,Bacharelado,Outros,Engenheiro,servicos,1.0,1.0,1.0,6.0,Outros,166.0,102.0,0.0,perfil4
15848,SEM FORMAÇÃO,UFRN,Supervisor,Siderurgica,0.0,0.0,1.0,8.0,Instagram,37.0,31.0,0.0,perfil2
15855,SEM FORMAÇÃO,UNIVERSIDADE NOVE DE JULHO,Advogado,Estado,0.0,0.0,1.0,10.0,Outros,102.0,93.0,1.0,perfil1
15856,Outros,FGV-RJ,Outros,Siderurgica,1.0,1.0,1.0,9.0,Outros,98.0,97.0,1.0,perfil1


In [15]:
#Gerando o arquivo para explorar os dados com o Power BI

df_training_dataset.to_csv('df_training_dataset_pre_processing.csv')

In [16]:
#Convertendo as variáveis para categóricas para ser usado na classificação

df_training = pd.get_dummies(df_training_dataset, columns=[
    'graduacao','universidade','organizacao','como_conheceu_lit','profissao'])
df_training.tail()

,pretende_fazer_cursos_lit,interesse_mba_lit,importante_ter_certificado,horas_semanais_estudo,modulos_iniciados,modulos_finalizados,certificados,categoria,graduacao_Bacharelado,graduacao_Especialização,...,profissao_Analista Senior,profissao_Assessor,profissao_Coordenador,profissao_Diretor,profissao_Engenheiro,profissao_Gerente,profissao_Outros,profissao_SEM EXPERIÊNCIA,profissao_Supervisor,profissao_Sócio/Dono/Proprietário
15844,0.0,0.0,1.0,6.0,39.0,32.0,0.0,perfil2,1,0,...,0,0,0,0,0,0,0,1,0,0
15847,1.0,1.0,1.0,6.0,166.0,102.0,0.0,perfil4,1,0,...,0,0,0,0,1,0,0,0,0,0
15848,0.0,0.0,1.0,8.0,37.0,31.0,0.0,perfil2,0,0,...,0,0,0,0,0,0,0,0,1,0
15855,0.0,0.0,1.0,10.0,102.0,93.0,1.0,perfil1,0,0,...,0,0,0,0,0,0,0,0,0,0
15856,1.0,1.0,1.0,9.0,98.0,97.0,1.0,perfil1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
#Dataset para classificação

df_training.to_csv('df_training_pre_processing_to_model.csv')